In [7]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [8]:

def getMaxContour(contours):
    maxArea = 0
    for c in contours:
        area = cv2.contourArea(c)
        
        if maxArea < area:
            maxArea = area
            maxContour = c
    positions = cv2.boundingRect(maxContour)
    return maxContour , positions

In [9]:
def limiar(img):
    gray = cv2.cvtColor(img,  cv2.COLOR_BGR2GRAY)
    gaussian = cv2.GaussianBlur(gray, (3, 3), 0)
    limiar, bin = cv2.threshold(gaussian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return bin

In [10]:
sources = ['Tomate_01.jpg','Tomate_02.jpg','Tomate_03.jpg','Tomate_04.jpg']
theshrold = [217, 299, 263, 255]
lower = np.array([0, 66, 168])
upper = np.array([14, 232, 255])
kernel = np.ones((5, 5), np.uint8)

def Process(source):
    
    path = 'source/'+source
    img = cv2.imread(path)
    b, g,r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])
    imgProcessed = rgb_img.copy()
    
    converter = img[282:282+59, 710 : 710+230]
    binConverter = limiar(converter)
    contoursConverter, _ = cv2.findContours(binConverter, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    _, posConverter = getMaxContour(contoursConverter)
    
    cv2.rectangle(converter, (posConverter[0]  , posConverter[1] ), (posConverter[0] + posConverter[2], posConverter[1] +posConverter[3]), (0, 0, 0), 2)  
#     cv2.putText(imgProcessed, ('Referencia'), (int(posConverter[0] + posConverter[2]/2), posConverter[1]),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)


    imgProcessed[282:282+59, 710 : 710+230] = converter

    th = theshrold[sources.index(source)]

    hsvImage = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsvImage, np.array(lower), np.array(upper))
    if sources.index(source)==0:
        mask = cv2.dilate(mask, kernel, iterations=1)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    plt.figure(figsize=(12, 9))
    plt.imshow(rgb_img)

    _, posTomate = getMaxContour(contours)
    cv2.rectangle(imgProcessed, (posTomate[0]  , posTomate[1] ), (posTomate[0] + posTomate[2], posTomate[1] +posTomate[3]), (0, 255, 0), 2)  
    
    tomate = rgb_img[posTomate[1] : posTomate[1]+posTomate[3], posTomate[0]: posTomate[0]+posTomate[2]]
    gray = cv2.cvtColor(tomate,  cv2.COLOR_RGB2GRAY )
    gauss =  cv2.GaussianBlur(gray, (5, 5), 0)     
    bin2 =  cv2.adaptiveThreshold(gauss ,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, th,2)
    elipseCont, _ = cv2.findContours(bin2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    cv2.putText(imgProcessed, ('Tomate'), (int(posTomate[0] + posTomate[2]/2), posTomate[1] -15),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

    
    for cont in elipseCont:
        elipseArea = cv2.contourArea(cont)
        if (elipseArea > 900 and elipseArea < 3000):
            pos = cv2.boundingRect(cont)
            w = (pos[2]*22)/ posConverter[2]
            h = (pos[3]*10)/ posConverter[3]
            print(posConverter)
            firmeza = (0.491)/(0.784*(w)*(h)/100) 
            cv2.rectangle(imgProcessed, (posTomate[0] + pos[0]  , posTomate[1] + pos[1]), (posTomate[0] + pos[0] + pos[2], posTomate[1] + pos[1]
                + pos[3]), (0, 0, 0), 2) 
            
            cv2.putText(imgProcessed, (f'Altura: {h/10:,.2f}'), (posTomate[0] + pos[0] + pos[2]+5, posTomate[1] + pos[1]+10),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
            cv2.putText(imgProcessed, (f'Largura: {w/10:,.2f}'), (posTomate[0] + pos[0] + pos[2]+5, posTomate[1] + pos[1]+20),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
            cv2.putText(imgProcessed, (f'Firmza: {firmeza:,.2f}'), (posTomate[0] + pos[0] + pos[2]+5, posTomate[1] + pos[1]+33),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    plt.figure(figsize=(12, 9))
    plt.imshow(imgProcessed)
    plt.show()
widgets.interact(Process, source = sources)



# Aumentar Tamanho
# Separar passos
# Por foto tirada pelo renan

interactive(children=(Dropdown(description='source', options=('Tomate_01.jpg', 'Tomate_02.jpg', 'Tomate_03.jpg…

<function __main__.Process(source)>